In [11]:
# Dependencies
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import calendar
import statsmodels.formula.api as sm
from sklearn.linear_model import LinearRegression
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [12]:
#load the full dataset and conduct data cleaning
# train_data=pd.read_csv('Resources/train.csv')

In [13]:
#descriptives of train_data to spot any thing that requires cleaning
#train_data.describe()

In [14]:
#drop the rows that fares are negative or equal to 0
#train_cleaned = train_data.drop(train_data[train_data['fare_amount']<=0].index)

#making sure all pick-up and drop-off are based inside New York City.
#New York City latitude and longitude range is: North Latitude: 40.917577 South Latitude: 40.477399 East Longitude: -73.700272 West Longitude: -74.259090
#Source: https://www.mapdevelopers.com/geocode_bounding_box.php
#Data out of the range will be cleaned out
#train_cleaned = train_cleaned.drop(train_cleaned[train_cleaned['pickup_longitude']>-73.700272].index)
#train_cleaned = train_cleaned.drop(train_cleaned[train_cleaned['pickup_longitude']<-74.259090].index)
#train_cleaned = train_cleaned.drop(train_cleaned[train_cleaned['dropoff_longitude']>-73.700272].index)
#train_cleaned = train_cleaned.drop(train_cleaned[train_cleaned['dropoff_longitude']<-74.259090].index)
#train_cleaned = train_cleaned.drop(train_cleaned[train_cleaned['pickup_latitude']>40.917577].index)
#train_cleaned = train_cleaned.drop(train_cleaned[train_cleaned['pickup_latitude']<40.477399].index)
#train_cleaned = train_cleaned.drop(train_cleaned[train_cleaned['dropoff_latitude']>40.917577].index)
#train_cleaned = train_cleaned.drop(train_cleaned[train_cleaned['dropoff_latitude']<40.477399].index)

In [15]:
#cleaned out the fare > 500
#train_cleaned = train_cleaned.drop(train_cleaned[train_cleaned['fare_amount']>500].index)
#train_cleaned.describe()

In [16]:
#cleaend out passenger_count > 9
#train_cleaned = train_cleaned.drop(train_cleaned[train_cleaned['passenger_count']>9].index)
#final clean data descriptives
#train_cleaned.describe()

In [17]:
#sample 25% of the training data
#sample_data = train_data.sample(frac=.25)
#sample_data.tail()
#sample_data.to_csv("Resources/sample.csv")
#sample_data.to_csv('Resources/sample_data_cleaned.csv', sep='\t')
#save the cleaned train data to csv
#train_cleaned.to_csv('train_data_cleaned.csv', sep='\t')

In [18]:
#Reading the cleaned 25% sample data from csv to use forward
data_df = pd.read_csv('Resources/sample_data_cleaned.csv')

# ------ TEMPORARY ----------
data_df = data_df.sample(frac=.25)

In [19]:
#calculate the distance between pick-up and drop-off spot
def distance(row):
    """
    Calculate the Haversine distance.

    Parameters
    ----------
    origin : tuple of float
        (lat, long)
    destination : tuple of float
        (lat, long)

    Returns
    -------
    distance_in_km : float

    Examples
    --------
    >>> origin = (48.1372, 11.5756)  # Munich
    >>> destination = (52.5186, 13.4083)  # Berlin
    >>> round(distance(origin, destination), 1)
    504.2
    """
    lat1, lon1 = row['pickup_latitude'], row['pickup_longitude']
    lat2, lon2 = row['dropoff_latitude'], row['dropoff_longitude']
    radius = 6371  # km

    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = (math.sin(dlat / 2) * math.sin(dlat / 2) +
         math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) *
         math.sin(dlon / 2) * math.sin(dlon / 2))
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = radius * c

    return d

In [ ]:
data_df.tail()

,Unnamed: 0,Unnamed: 0.1,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
13756128,13804748,49984689,2009-10-05 19:13:00.00000028,12.5,2009-10-05 19:13:00 UTC,-73.982347,40.762462,-73.998173,40.722813,2
6674690,6698322,22878835,2012-06-25 07:18:26.0000002,8.9,2012-06-25 07:18:26 UTC,-73.984772,40.724445,-73.992033,40.748923,1
8880091,8911582,3511857,2012-09-07 23:43:23.0000002,12.0,2012-09-07 23:43:23 UTC,-73.984412,40.761077,-73.998875,40.732322,2
8303661,8333107,5383332,2010-06-29 20:17:23.0000007,4.9,2010-06-29 20:17:23 UTC,-73.975040,40.750105,-73.983394,40.743880,1
5156399,5174572,34394456,2011-01-12 11:21:00.00000063,7.3,2011-01-12 11:21:00 UTC,-73.986378,40.743072,-73.996405,40.759798,5


In [ ]:
data_df['new_time'] = (data_df['pickup_datetime']).apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S %Z"))    
data_df = data_df.drop(['Unnamed: 0','key','pickup_datetime'], axis=1)
time_range = (data_df['new_time'] > "2013-01-01") & (data_df['new_time'] < "2016-12-31")
ranged_data = data_df.loc[time_range]

In [ ]:
ranged_data['distance'] = data_df.apply(distance, axis=1)

In [ ]:
ranged_data['year'] = data_df['new_time'].apply(lambda x: x.year)
ranged_data['hour_of_day'] = data_df['new_time'].apply(lambda x: x.hour)
ranged_data['day'] = data_df['new_time'].apply(lambda x: calendar.day_name[x.weekday()])
ranged_data['date'] = data_df['new_time'].apply(lambda x: x.date())

#plt.scatter(ranged_data['hour_of_day'],ranged_data['fare_amount'])

ranged_data = ranged_data[ranged_data['distance'] < 100]


In [ ]:
dummied_data = ranged_data

for column in dummied_data.columns:
 if dummied_data[column].dtype==object:
  dummyCols=pd.get_dummies(dummied_data[column])
  dummied_data=dummied_data.join(dummyCols)
  del dummied_data[column]

fares = dummied_data['fare_amount']
dummied_data.drop(['fare_amount','pickup_longitude', 'pickup_latitude','dropoff_longitude','new_time'], axis=1,inplace=True)


model_1=LinearRegression()
model_1.fit(dummied_data,fares)
coefficients=pd.DataFrame({'name':list(dummied_data),'value':model_1.coef_})
print("Mean squared error:",np.mean((model_1.predict(dummied_data) - fares) ** 2))
print("R²:",model_1.score(dummied_data, fares))

In [ ]:
ranged_data = ranged_data.drop(ranged_data[ranged_data['passenger_count'] == 0].index)
ranged_data.sort_values(by='passenger_count')

In [ ]:
by_distance=ranged_data.groupby(by='hour_of_day')
#x = x.drop(x['day'] == 'Sunday')
x = by_distance.agg('mean')['fare_amount']
plt.bar(x.index,x)

In [ ]:
plt.title("Hour of Day vs Fare Amount")
plt.xlabel("Hour of Day ")
plt.ylabel("Average Fair")
plt.show()

In [ ]:
plt.scatter(ranged_data['distance'],ranged_data['fare_amount'])
plt.ylim(0,500)

plt.title("Distance vs Fare Amount")
plt.xlabel("Distance (miles)")
plt.ylabel("Fair Amount")


plt.show()


In [ ]:
by_distance=ranged_data.groupby(by='day')
#x = x.drop(x['day'] == 'Sunday')
x = by_distance.agg('mean')['fare_amount']
width = 0.4
plt.bar(x.index, x, width)

In [ ]:
plt.title("Mean of Fare Amounts vs Day")
plt.xlabel("Days of the Week")
plt.ylabel("Avg Fair Amount")
plt.show()

In [ ]:
by_distance=ranged_data.groupby(by='year')
x = by_distance.agg('mean')['fare_amount']
width = 0.5
plt.bar(x.index, x, width)

In [ ]:

plt.title("Mean Fare Amount vs Year")
plt.xlabel("Years")
plt.ylabel("Avg Fare Amount")
plt.xticks(x.index, ('2013','2014', '2015'))


In [ ]:

plt.show()

In [ ]:

x.index

In [ ]:

width = 3.0
plt.bar(x.index, x, width, color = "blue")
plt.title("Passenger Count vs Mean Fare Amount")
plt.xlabel("Passenger Count")
plt.ylabel("Avg Fare Amount")

plt.show()